# Преобразования данных о недвижимости

В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

*  удалили столбцы `index` и `Coordinates`;
*  создали признак средней площади одной комнаты `MeanRoomsSquare` и ввели коэффициент соотношения площади здания к площади участка — `AreaRatio`;
* преобразовали признак даты продажи `Date` в формат `datetime` и создали на его основе следующие столбцы: номер месяц продажи (`MonthSale`), номер дня недели продажи (`WeekdaySale`), «признак-мигалку» выходного дня (`Weekend`);
* заменили признак года постройки здания `YearBuilt` на его возраст `AgeBuilding`;
извлекли из признака адреса объекта `Address` новый признак подтипа улицы `StreetType` и удалили столбец с адресом;
уменьшили число уникальных наименований агентств по недвижимости (`SellerG`), а также число пригородов (`Suburb`);
выделили категориальные признаки и преобразовали их в тип данных `сategory`;
* заменили сокращённые названия категорий признака типа объекта `Type` на их полные названия (h — house, t —  townhouse, u — unit).

## 1.1

Преобразуйте столбец `Date` в формат `datetime` и выделите квартал (`quarter`) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [137]:
import pandas as pd

melb_df = pd.read_csv("data/melb_data_fe.csv")
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
print(melb_df.Date.dt.quarter.value_counts())

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64


## 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы `Date`, `Rooms`, `Bedroom`, `Bathroom`, `Car`.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных `category`.

In [138]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_unique_count = 150 
for col in melb_df.columns: 
    if (melb_df[col].nunique() < max_unique_count and 
        col not in cols_to_exclude): 
        melb_df[col] = melb_df[col].astype('category')
        
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

# Сортировка

`sort_values()` - метод

Параметры:

* `by` - имя столбца или список столбцов, по которым производится сортировка;

*  `axis=0` ось по которой производится сортировка (по умолчанию по строкам = 0);

* `ascending=True` - от меньшего к большему;

* `ingnore_index=False` - сохраняет исходные индексы таблицы;

* `inplace=False` - сортировать на месте (изменение исходной) или вернуть сортированную копию.

Комбинирование сортировки с фильтрацией

Найдём информацию о таунхаусах (`Type`), проданных компанией (`SellerG`) `McGrath`, у которых коэффициент соотношения площадей здания и участка (`AreaRatio`) меньше -0.8 (у которых площадь участка существенно больше площади здания). Результат отсортируем по дате продажи (`Date`) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр `ignore_index` на `True`. Для наглядности результата выберем из таблицы только столбцы `Data` и `AreaRatio`:



In [139]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


## 2.2

Произведите сортировку столбца `AreaRatio` по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке `1558`? Ответ округлите до целого числа.



In [140]:
melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558, 'BuildingArea']

126.0

## 2.3

Найдите таунхаусы (`Type`) с количеством жилых комнат (`Rooms`) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (`MeanRoomsSquare`). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [141]:
mask1 = melb_df.Type == 'townhouse'
mask2 = melb_df.Rooms > 2
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price']

1300000.0

# Группировка

Метод `groupby()`

Параметры метода:

* `by` — имя или список имён столбцов, по которым производится группировка.
* `axis` — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
* `as_index` — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на `True`.

возвращает объект `DataFrameGroupBy`, который хранит в себе информацию о том, какие строки относятся к определённой группе.

к этому объекту можно применять агрегирующие методы (`mean`, `median`, `sum` и т. д.), чтобы рассчитывать показатели внутри каждой группы.



In [142]:
# Пример: удаленность района от центра

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

Метод `agg()`

используется для расчета нескольких агрегирующих методов.

In [143]:
# пример использования метода agg

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [144]:
# полная информация обо всех статистических методах

melb_df.groupby('MonthSale')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


Метод `agg` с другими функциями. 

Передадим дополнительно встроенную функцию `set`, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов

In [145]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{Jellis, hockingstuart, McGrath, Buxton, Harco..."
Eastern Victoria,11,"{hockingstuart, Ray, McGrath, Barry, Eview, ot..."
Northern Metropolitan,40,"{Collins, Jellis, hockingstuart, McGrath, Swee..."
Northern Victoria,11,"{hockingstuart, McGrath, Ray, Barry, other, Mc..."
South-Eastern Metropolitan,25,"{Jellis, hockingstuart, Buxton, McGrath, Eview..."
Southern Metropolitan,38,"{Collins, Jellis, hockingstuart, Buxton, McGra..."
Western Metropolitan,34,"{Jellis, hockingstuart, Sweeney, McGrath, Harc..."
Western Victoria,6,"{hockingstuart, Ray, other, YPA, Raine, HAR}"


## 3.1

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [146]:
melb_df.groupby(by='Rooms')['Price'].mean().sort_values(
    ascending=False).index[0]

7

## 3.2

Какой регион имеет наименьшее стандартное отклонение по географической широте (`Lattitude`)?
В качестве ответа запишите название этого региона.

In [147]:
melb_df.groupby(by='Regionname')['Lattitude'].std().sort_values().index[0]

'Western Victoria'

## 3.3

Какая риелторская компания (`SellerG`) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
Для ответа на этот вопрос рассчитайте сумму продаж (`Price`) каждой компании в заданный период.
Не забудьте перевести даты в формат `datetime`.

In [148]:
mask1 = melb_df['Date'] <= pd.to_datetime('2017-09-01') 
mask2 = melb_df['Date'] >= pd.to_datetime('2017-05-01')
melb_df[mask1 & mask2].groupby(by='SellerG')['Price'].sum().sort_values().index[0]


'LITTLE'

# Сводные таблицы

Создание сводной таблицы при помощи метода `groupby()`

In [149]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().round()

Rooms  Type     
1      house         866866.0
       townhouse     592705.0
       unit          389929.0
2      house        1017238.0
       townhouse     710158.0
       unit          610491.0
3      house        1109233.0
       townhouse     984709.0
       unit          850596.0
4      house        1462283.0
       townhouse    1217092.0
       unit         1037476.0
5      house        1877327.0
       townhouse    1035000.0
       unit               NaN
6      house        1869508.0
       townhouse          NaN
       unit          520000.0
7      house        1920700.0
       townhouse          NaN
       unit               NaN
8      house        1510286.0
       townhouse          NaN
       unit         2250000.0
10     house         900000.0
       townhouse          NaN
       unit               NaN
Name: Price, dtype: float64

Метод `unstack()`

используется, чтобы переопределить вложенные индексы как столбцы.

In [150]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


## Метод `pivot_table()`

Параметры метода:

* `values` — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
* `index` — имя столбца, значения которого станут строками сводной таблицы;
* `columns` — имя столбца, значения которого станут столбцами сводной таблицы;
* `aggfunc` — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего -`mean`);
* `fill_value` — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

In [151]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [152]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [153]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

## Многомерные сводные таблицы

In [154]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

## Доступ к данные сводной таблицы

In [155]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
display(pivot)

median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

In [156]:
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [157]:
display(pivot['mean']['unit'])

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

## Фильтрация сводной таблицы

In [158]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [159]:
print(list(filtered_pivot.index))
# ['Southern Metropolitan', 'Western Metropolitan']

['Southern Metropolitan', 'Western Metropolitan']


## Создание мультииндексных DataFrame

In [160]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

white  up       0.642774
       down     0.409496
       right    0.531610
blue   up       0.670541
       down     0.978163
red    up       0.938237
       down     0.611860
       left     0.870343
dtype: float64

In [161]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up    0.265217  0.185228 -1.094579 -0.132195
      down -0.878841  0.231540  0.806854 -0.467771
red   up    0.386840  0.319460  0.775675 -0.025539
      down -0.507862 -0.056318 -1.611244 -0.293077

## 4.2

Составьте сводную таблицу, которая показывает зависимость медианной площади (`BuildingArea`) здания от типа объекта недвижимости (`Type`) и количества жилых комнат в доме (`Rooms`). Для какой комбинации признаков площадь здания наибольшая?
В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

In [162]:
area_pt = melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc='median'
)

display(area_pt)

Type,house,townhouse,unit
Rooms,,,
1,126.0,88.0,69.5
2,126.0,114.0,110.0
3,126.0,126.0,126.0
4,141.0,159.5,126.0
5,177.0,152.0,NaN
6,126.0,NaN,171.0
7,216.5,NaN,NaN
8,126.0,NaN,126.0
10,126.0,NaN,NaN


In [163]:
type_building = area_pt.max().index[0]
num_rooms = area_pt[area_pt[type_building] == area_pt.max()[0]].index[0]
print(f"Result: {type_building},{num_rooms}")



Result: house,7


## 4.3

Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (`Price`) от риелторского агентства (`SellerG`) и типа здания (`Type`).
Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа `unit` максимальна. В качестве ответа запишите название этого агентства.

In [164]:
price_pt = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median' 
)

display(price_pt)

Type,house,townhouse,unit
SellerG,,,
Alexkarbon,1036250.0,935000.0,560000.0
Barry,830000.0,727500.0,502000.0
Bells,692500.0,430500.0,457500.0
Biggin,1151750.0,675000.0,548000.0
Brad,875000.0,700000.0,465000.0
Buckingham,830000.0,820000.0,553000.0
Burnham,922500.0,NaN,305000.0
Buxton,1323750.0,1030000.0,670000.0
C21,836500.0,768750.0,625500.0


In [165]:
max_price_unit = price_pt['unit'].max()
name_of_seller = price_pt[price_pt['unit'] == max_price_unit].index[0]

print(f"The seller '{name_of_seller}' has the max median price value "
      f"for the building type 'unit', which is {round(max_price_unit)} "
      f"Australian dollars.")

The seller 'Nick' has the max median price value for the building type 'unit', which is 900000 Australian dollars.


# Новые данные (оценка фильмов)

* `ratings1` и `ratings2` — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.

    * `userId` — уникальный идентификатор пользователя, который выставил оценку;

    * `movieId` — уникальный идентификатор фильма;

    * `rating` — рейтинг фильма.

* `dates` — таблица с датами выставления всех оценок.

    * `date` — дата и время выставления оценки фильму.

* `movies` — таблица с информацией о фильмах.

    * `movieId` — уникальный идентификатор фильма;

    * `title` — название фильма и год его выхода;

    * `genres` — жанры фильма.



## 5.2

Сколько уникальных фильмов представлено в таблице `movies`?

In [166]:
movies = pd.read_csv('data/movies.csv')
movies.shape

(9742, 3)

## 5.3

Сколько уникальных пользователей в таблице `ratings1`?

In [167]:
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
ratings1['userId'].nunique()

274

## 5.4

В каком году было выставлено больше всего оценок?

Для ответа на этот вопрос используйте таблицу `dates`.

In [168]:
dates = pd.read_csv('data/dates.csv')
dates['date'] = pd.to_datetime(dates['date'])

In [169]:
years_series = dates['date'].dt.year
years_series.value_counts().index[0]

2000

# Объединение `DataFrame`: `concat()`

[Статья про объединение (дополнительно)](https://pythobyte.com/how-to-merge-dataframes-in-pandas-23614/)

Функция `pandas.concat()`

* `objs` — список объектов DataFrame (`[df1, df2,…]`), которые должны быть сконкатенированы;

* `axis` — ось определяет направление конкатенации: `0` — конкатенация по строкам (по умолчанию), `1` — конкатенация по столбцам;

* `join` — либо `inner` (пересечение), либо `outer` (объединение); рассмотрим этот момент немного позже;

* `ignore_index` — по умолчанию установлено значение `False`, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение `True`, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

In [170]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [171]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [172]:
display(ratings1.tail(1))
display(ratings2.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


In [173]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [174]:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


# Объединение `DataFrame`: `join()`, `merge()`



## Типы объединений

* `inner` - внутреннее<br>
При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.

* `outer` - внешнее

    * `full` — используется как `outer` по умолчанию, объединяет все варианты в обеих таблицах.

    * `left` — для всех записей из «левой» таблицы (например, `ratings`) ведётся поиск соответствий в «правой» (например, `movies`). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.

    * `right` — аналогично предыдущему, но остаются значения только из «правой» таблицы.



## Метод `join()`

* `other` — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».

* `how` — параметр типа объединения. Он может принимать значения `'inner'`, `'left'` (left outer), `'right'` (right outer), и `'outer'` (full outer). По умолчанию параметр установлен на `'left'`.

* `on` — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».

* `lsuffix` и `rsuffix` — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

In [194]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())
display(joined.tail())
display(joined.title.value_counts())
display(movies[movies.title == 'Forrest Gump (1994)'])

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


,userId,movieId,rating,date,title,genres
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi
100835,610,170875,3.0,2017-05-03 21:20:15,The Fate of the Furious (2017),Action|Crime|Drama|Thriller


Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
                                   ... 
Sex, Drugs & Taxation (2013)          1
Extraordinary Tales (2015)            1
Tomorrow (2015)                       1
Embrace of the Serpent (2016)         1
31 (2016)                             1
Name: title, Length: 9719, dtype: int64

,movieId,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


## Метод `merge()`

* `right` — присоединяемая таблица. По умолчанию она является «правой».

* `how` — параметр типа объединения. По умолчанию принимает значение `'inner'`.

* `on` — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.

* `left_on` — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.

* `right_on` — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.

In [195]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [196]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


Метод `merge()` с внешним (`'outer'`) типом объединения может использоваться как аналог метода `concat()` при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.

__Дубликаты будут удаляться автоматически__

In [197]:
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

Число строк в таблице merge_ratings:  100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0
